In [ ]:
## CCI Assessor

import glob

complete = '/gws/nopw/j04/esacci_portal/kerchunk_conversion/complete/'

with open('/gws/nopw/j04/esacci_portal/kerchunk_conversion/groups/cci_group_v1/blacklist_codes.txt') as f:
    blacklist = [r.split(',')[0] for r in f.readlines()]

revalidate = []

types = glob.glob('./*.csv')
for t in types:
    with open(t) as f:
        proj_codes = [r.split(',')[0] for r in f.readlines()]
    sumt = 0
    for proj_code in proj_codes:
        if len(glob.glob(f'{complete}/cci_group_v1/{proj_code}*')) > 0:
            sumt += 1
        elif proj_code in blacklist:
            sumt += 1
        else:
            if 'cci_group' not in t:
                revalidate.append(proj_code)
        
    if sumt == len(proj_codes):
        print(f'{t} - complete')
    else:
        print(f'{t} - {sumt} vs {len(proj_codes)}')


In [1]:
import glob
complete = '/gws/nopw/j04/cmip6_prep_vol1/kerchunk-pipeline/complete/CMIP6_rel1_6233'
in_progress = '/gws/nopw/j04/cmip6_prep_vol1/kerchunk-pipeline/in_progress/CMIP6_rel1_6233'

completes = glob.glob(f'{complete}/*.json')
proj_codes = [c.split('kr1.')[0][:-1] for c in completes]

In [8]:
in_progress = '/gws/nopw/j04/cmip6_prep_vol1/kerchunk-pipeline/in_progress/CMIP6_rel1_6233'

In [10]:
import os
for p in proj_codes:
    proj_code = p.split('/')[-1].replace('CMIP6_','')
    newlog = f'{in_progress}/{proj_code}/status_log.csv'
    os.system(f"touch {newlog}")
    with open(f"{newlog}",'w') as f:
        f.write('complete,complete,12:07 03/06/24,,False')

In [15]:
## Fix statistics issues

workdir = '/gws/nopw/j04/esacci_portal/kerchunk_conversion/'

groups = [
    'aerosol_v1',
    'cci_group_v1',
    'cloud_v1',
    'land_cover_v1',
    'reccap_v1',
    'salinity_v1',
    'sea_level_v1',
    'seaice_v1',
    'snow_v1'
]

In [1]:
def get_codes(group, filename):
    with open(f'{workdir}/groups/{group}/{filename}.txt') as f:
        return [r.strip() for r in f.readlines()]

In [2]:
def format_float(value: int, logger):
    """Format byte-value with proper units"""
    logger.debug(f'Formatting value {value} in bytes')
    if value:
        unit_index = 0
        units = ['','K','M','G','T','P']
        while value > 1000:
            value = value / 1000
            unit_index += 1
        return f'{value:.2f} {units[unit_index]}B'
    else:
        return None

In [9]:
from pipeline.logs import FalseLogger
from pipeline.scan import format_float
import json

workdir = '/gws/nopw/j04/cmip6_prep_vol1/kerchunk-pipeline'

def fix_scan(group, proj):
    try:
        with open(f'{workdir}/in_progress/{group}/{proj}/detail-cfg.json') as f:
            detail = json.load(f)
    except FileNotFoundError:
        return 0
    except json.JSONDecodeError:
        return 0

    if 'skipped' in detail:
        return 0

    detail['kerchunk_data'] = format_float(float(detail['chunks_per_file']) * int(detail['num_files']) * 167, FalseLogger())
    if 'B' not in detail['data_represented']:
        detail['netcdf_data'] = format_float(float(detail['data_represented']), FalseLogger())
    else:
        detail['netcdf_data'] = detail['data_represented']
    
    with open(f'{workdir}/in_progress/{group}/{proj}/detail-cfg.json','w') as f:
        f.write(json.dumps(detail))
    return 1

corrected = 0
for g in ['CMIP6_rel1_6233']:
    projs = get_codes(g, 'proj_codes_1')
    for p in projs:
        corrected += fix_scan(g, p)
corrected

5579

In [25]:
def mem_to_val(value):
    suffixes = {
        'KB': 1000,
        'MB': 1000000,
        'GB': 1000000000,
        'TB': 1000000000000,
        'PB': 1000000000000000}
    suff = suffixes[value.split(' ')[1]]
    return float(value.split(' ')[0]) * suff

In [32]:
import json
import glob
import os


def analyse_data(g):
    print(g)
    ncf, ker = 0, 0
    kus = 0
    complete, scanned = 0, 0
    kerarr = []
    projset = glob.glob(f'/gws/nopw/j04/esacci_portal/kerchunk_conversion/in_progress/{g}/*')
    for p in projset:
        details = None
        try:
            with open(f'{p}/detail-cfg.json') as f:
                details = json.load(f)
        except:
            pass
        if details:
            scanned += 1
            if 'netcdf_data' in details:
                ncf += mem_to_val(details['netcdf_data'])
                ker += mem_to_val(details['kerchunk_data'])
                kerarr.append((details['netcdf_data'], details['kerchunk_data']))
                if os.path.isfile(f'{p}/kerchunk-1a.json.complete'):
                    kus += mem_to_val(details['kerchunk_data'])
                    complete += 1
    return ncf, ker, kus, scanned, complete

In [34]:
Tncf, Tker, Tkus, Tscan, Tcomp = 0, 0, 0, 0, 0
for g in groups:
    ncf, ker, kus, scanned, complete = analyse_data(g)
    print(f'         NetCDF: {format_float(ncf, FalseLogger())}')
    print(f'  Kerchunk Estm: {format_float(ker, FalseLogger())} ({scanned})')
    print(f'Kerchunk Actual: {format_float(kus, FalseLogger())} ({complete})')
    print()

    Tncf += ncf
    Tker += ker
    Tkus += kus
    Tscan += scanned
    Tcomp += complete

print('Total CCI Deliverable Summary')
print(f'         NetCDF: {format_float(Tncf, FalseLogger())}')
print(f'  Kerchunk Estm: {format_float(Tker, FalseLogger())} ({Tscan})')
print(f'Kerchunk Actual: {format_float(Tkus, FalseLogger())} ({Tcomp})')

aerosol_v1
         NetCDF: 675.60 GB
  Kerchunk Estm: 2.00 GB (38)
Kerchunk Actual: 339.48 MB (26)

cci_group_v1
         NetCDF: 59.84 TB
  Kerchunk Estm: 1.58 TB (311)
Kerchunk Actual: 5.37 GB (128)

cloud_v1
         NetCDF: 111.41 GB
  Kerchunk Estm: 49.75 MB (24)
Kerchunk Actual: 49.75 MB (24)

land_cover_v1
         NetCDF: 112.27 GB
  Kerchunk Estm: 209.68 MB (3)
Kerchunk Actual: 209.68 MB (2)

reccap_v1
         NetCDF: None
  Kerchunk Estm: None (1)
Kerchunk Actual: None (0)

salinity_v1
         NetCDF: 79.37 GB
  Kerchunk Estm: 38.60 MB (10)
Kerchunk Actual: 38.60 MB (10)

sea_level_v1
         NetCDF: 2.77 GB
  Kerchunk Estm: 5.23 MB (7)
Kerchunk Actual: None (0)

seaice_v1
         NetCDF: 253.43 GB
  Kerchunk Estm: 92.47 MB (16)
Kerchunk Actual: 78.86 MB (12)

snow_v1
         NetCDF: 767.62 GB
  Kerchunk Estm: 1.71 GB (10)
Kerchunk Actual: 1.71 GB (10)

Total CCI Deliverable Summary
         NetCDF: 61.84 TB
  Kerchunk Estm: 1.59 TB (420)
Kerchunk Actual: 7.79 GB (212)


In [42]:
kerarr

[('68.01 GB', '8.34 GB'),
 ('70.91 GB', '8.34 GB'),
 ('262.08 GB', '774.76 MB'),
 ('151.04 GB', '25.46 GB'),
 ('119.88 GB', '53.88 MB'),
 ('210.22 GB', '68.97 MB'),
 ('214.39 GB', '68.97 MB'),
 ('7.18 GB', '721.44 KB'),
 ('22.31 GB', '9.51 GB'),
 ('20.31 GB', '9.51 GB'),
 ('318.24 GB', '917.67 MB'),
 ('183.96 GB', '27.77 GB'),
 ('462.00 GB', '1.40 GB'),
 ('269.04 GB', '42.59 GB'),
 ('107.16 GB', '14.38 GB'),
 ('110.48 GB', '14.38 GB'),
 ('124.91 GB', '17.63 GB'),
 ('163.34 GB', '17.63 GB'),
 ('624.98 GB', '1.80 GB'),
 ('324.30 GB', '54.38 GB'),
 ('18.39 GB', '9.44 GB'),
 ('17.61 GB', '9.44 GB'),
 ('466.10 GB', '2.54 GB'),
 ('41.46 GB', '15.28 GB'),
 ('914.40 GB', '4.91 GB'),
 ('79.52 GB', '29.80 GB'),
 ('80.60 GB', '12.08 GB'),
 ('80.68 GB', '12.08 GB'),
 ('384.54 GB', '1.13 GB'),
 ('204.20 GB', '33.35 GB'),
 ('43.17 GB', '16.44 GB'),
 ('42.30 GB', '16.44 GB'),
 ('907.12 GB', '2.58 GB'),
 ('459.37 GB', '80.14 GB'),
 ('128.55 GB', '18.25 GB'),
 ('133.59 GB', '18.25 GB'),
 ('642.46 GB', 

In [89]:
import glob
import json
from datetime import datetime 

reason = 'Archived at kr1.1 already'

for kfile in glob.glob('/gws/nopw/j04/esacci_portal/kerchunk_conversion/needs_version_updating/*'):
    old_version = 'kr1.1' # Hardcoded for now
    os.system(f"mv {kfile} {kfile.replace(old_version, 'kr1.2')}")

mv: '/gws/nopw/j04/esacci_portal/kerchunk_conversion/needs_version_updating/ESACCI-L3C_SNOW-SWE-MERGED-19790102-20200524-fv2.0_kr1.2.json' and '/gws/nopw/j04/esacci_portal/kerchunk_conversion/needs_version_updating/ESACCI-L3C_SNOW-SWE-MERGED-19790102-20200524-fv2.0_kr1.2.json' are the same file


In [7]:
len(revalidate)

25

In [2]:
fileset = """/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2003-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2004-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2005-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2006-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2007-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2008-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2009-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2010-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2011-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2012-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2013-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2014-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2015-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2016-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2017-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2018-fv03.0.nc
/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2019-fv03.0.nc""".split('\n')
fileset

['/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2003-fv03.0.nc',
 '/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2004-fv03.0.nc',
 '/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2005-fv03.0.nc',
 '/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2006-fv03.0.nc',
 '/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2007-fv03.0.nc',
 '/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2008-fv03.0.nc',
 '/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0/ESACCI-PERMAFROST-L4-PFR-MODISLST_CRYOGRID-AREA4_PP-2009-fv03.0.nc',
 '/neodc/esacci/permafrost/data/permafros

In [12]:
completes = os.walk(complete)

In [15]:
fileset = []
for roots, dirs, files in completes:
    fileset = files

In [50]:
byhand = """ESACCI-OC-L3S-K_490-MERGED-8D_DAILY_4km_SIN_PML_K_490_Lee-199709-202212-fv6.1
ESACCI-LST-L3S-LST-IRMGP_-0.05deg_1MONTHLY-20090101150000-20201201150000-fv1.00
ESACCI-LST-L3S-LST-IRMGP_-0.05deg_1MONTHLY-20090101180000-20201201180000-fv1.00
ESACCI-LST-L3S-LST-IRMGP_-0.05deg_1MONTHLY-20090101210000-20201201210000-fv1.00
ESACCI-OC-L3S-RRS-MERGED-1M_MONTHLY_4km_SIN_PML_RRS-199709-202212-fv6.0
ESACCI-OC-L3S-IOP-MERGED-1M_MONTHLY_4km_SIN_PML_OCx_QAA-199709-202212-fv6.0
ESACCI-OC-L3S-CHLOR_A-MERGED-8D_DAILY_4km_GEO_PML_OCx-19970829-20221227-fv6.0
ESACCI-OC-L3S-K_490-MERGED-5D_DAILY_4km_SIN_PML_K_490_Lee-199709-202212-fv6.2
ESACCI-OC-L3S-CHLOR_A-MERGED-5D_DAILY_4km_SIN_PML_OCx-19970903-20221227-fv6.0
ESACCI-OC-L3S-CHLOR_A-MERGED-8D_DAILY_4km_SIN_PML_OCx-19970829-20221227-fv6.0
ESACCI-OC-L3S-K_490-MERGED-8D_DAILY_4km_GEO_PML_K_490_Lee-199709-202212-fv6.1
ESACCI-LST-L3C-LST-SLSTRA-0.01deg_1DAILY_DAY-20160501-20201231-fv3.00.nc
ESACCI-LST-L3C-LST-ATSR_2-0.01deg_1DAILY_DAY-19950801-20030622-fv3.00
ESACCI-LST-L3C-LST-ATSR_2-0.01deg_1DAILY_NIGHT-19950801-20030622-fv3.00
ESACCI-LST-L3C-LST-ATSR_3-0.01deg_1DAILY_DAY-20020725-20120408-fv3.00
ESACCI-OC-L3S-CHLOR_A-MERGED-5D_DAILY_4km_GEO_PML_OCx-19970903-20221227-fv6.0
ESACCI-LST-L3C-LST-SLSTRA-0.01deg_1DAILY_NIGHT-20160501-20201231-fv3.00.nc
ESACCI-OC-L3S-OC_PRODUCTS-MERGED-1M_MONTHLY_4km_SIN_PML_OCx_QAA-199709-202212-fv6.0
ESACCI-OC-L3S-K_490-MERGED-5D_DAILY_4km_GEO_PML_K_490_Lee-199709-202212-fv6.2
ESACCI-OC-L3S-CHLOR_A-MERGED-1Y_YEARLY_4km_GEO_PML_OCx-1997-2022-fv6.0
ESACCI-OC-L3S-IOP-MERGED-1Y_YEARLY_4km_GEO_PML_OCx_QAA_Lee-199709-202212-fv6.1""".split('\n')

In [16]:
fileset

['ESACCI-L3C_SNOW-SCFG-AATSR_ENVISAT-20020520-20120408-fv1.0_kr1.0.json',
 'ESACCI-L3C_SNOW-SCFG-ATSR-2_ERS-2-19950801-20030622-fv1.0_kr1.0.json',
 'ESACCI-L3C_SNOW-SCFG-AVHRR_MERGED-19820101-20181230-fv2.0_kr1.0.json',
 'ESACCI-L3C_SNOW-SCFG-MODIS_TERRA-20000224-20201231-fv2.0_kr1.0.json',
 'ESACCI-L3C_SNOW-SCFV-AATSR_ENVISAT-20020520-20120408-fv1.0_kr1.0.json',
 'ESACCI-L3C_SNOW-SCFV-ATSR-2_ERS-2-19950801-20030622-fv1.0_kr1.0.json',
 'ESACCI-L3C_SNOW-SCFV-AVHRR_MERGED-19820101-20181230-fv2.0_kr1.0.json',
 'ESACCI-L3C_SNOW-SCFV-MODIS_TERRA-20000224-20201231-fv2.0_kr1.0.json',
 'ESACCI-L3C_SNOW-SWE-MERGED-19790102-20200524-fv2.0_kr1.2.json',
 'ESACCI-L3S_SNOW-SCFG-MERGED-CRYOCLIM-19820101-2019_kr1.0.json']

In [76]:
## Assessor 2
import os

tryset = """20210226-ESACCI-L4_AIS-GMB-BASIN-fv3.0
20210226-ESACCI-L3C_AIS-GMB-GRID-fv3.0
20170101-20200801-ESACCI-L3C_AIS-IV-S1-1M_200m-fv1.0.nc""".split('\n')

groups = """aerosol_v1
cci_group_v1
cloud_v1
land_cover_v1
reccap_v1
salinity_v1
seaice_v1
snow_v1""".split('\n')

for t in tryset:
    if t in byhand:
        print(t, 'byhand')
    for g in groups:
        if len(glob.glob(f'{complete}/{g}/{t}*')) > 0:
            print(t, 'complete')

20210226-ESACCI-L4_AIS-GMB-BASIN-fv3.0 complete
20210226-ESACCI-L3C_AIS-GMB-GRID-fv3.0 complete


In [35]:
tryset

['ESACCI-OC-L3S-K_490-MERGED-1Y_YEARLY_4km_SIN_PML_K_490_Lee-199709-202212-fv6.1',
 'ESACCI-OC-MAPPED-CLIMATOLOGY-1M_MONTHLY_4km_PML_OCx_QAA-01-12-fv6.0.nc',
 'ESACCI-OZONE-L3-LP-MEGRIDOP_MERGED-20011101_20221201-fv0004',
 'ESACCI-OZONE-L3-LP-SAGE_CCI_OMPS_plus_MERGED-19841001_20211201-fv0007',
 'ESACCI-OZONE-L3-LP-MERGED-MZM-200701-200812-fv0002']

In [8]:
ds.polar_stereographic

<xarray.DataArray 'polar_stereographic' (time: 17)>
array([-2147483647, -2147483647, -2147483647, -2147483647, -2147483647,
       -2147483647, -2147483647, -2147483647, -2147483647, -2147483647,
       -2147483647, -2147483647, -2147483647, -2147483647, -2147483647,
       -2147483647, -2147483647], dtype=int32)
Coordinates:
  * time     (time) datetime64[ns] 2003-01-01 2004-01-01 ... 2019-01-01
Attributes: (12/13)
    grid_mapping_name:                      polar_stereographic
    straight_vertical_longitude_from_pole:  0.0
    false_easting:                          0.0
    false_northing:                         0.0
    latitude_of_projection_origin:          90.0
    standard_parallel:                      71.0
    ...                                     ...
    semi_major_axis:                        6378137.0
    inverse_flattening:                     298.257223563
    spatial_ref:                            GEOGCS["WGS 84",DATUM["WGS_1984",...
    crs_wkt:                                PROJCS["WGS 84 / Arctic Polar Ste...
    esri_wkt:                               PROJCS["WGS_84_Arctic_Polar_Stere...
    GeoTransform:                           -8679599.425969256 926.6254331383...

In [11]:
dsx.isel(time=0).time.shape

AttributeError: 'Dataset' object has no attribute 'time'

In [11]:
import json
update = ['ESACCI-L3C_SNOW-SWE-MERGED-19790102-20200524-fv2.0']
group = 'snow_v1'

for proj_code in update:
    detailfile = f'/gws/nopw/j04/esacci_portal/kerchunk_conversion/in_progress/{group}/{proj_code}/detail-cfg.json'
    with open(detailfile) as f:
        details = json.load(f)
    details['version_no'] = 'kr1.1'
    with open(detailfile,'w') as f:
        f.write(json.dumps(details))
print('ding')

ding


In [ ]:
import glob
import json

checkfile = f'/gws/nopw/j04/esacci_portal/kerchunk_conversion/in_progress/cci_group_v1'
proj_codes = glob.glob(checkfile)

with open('/gws/nopw/j04/esacci_portal/kerchunk_conversion/input_files/cci_group_v1.csv') as f:
    origin = [r.strip() for r in f.readlines()]

for x, entry in enumerate(origin):
    row = entry.split(',')
    path = f'{checkfile}/{row[0]}'
    try:
        with open(f'{path}/allfiles.txt') as f:
            length = len(f.readlines())
        assert length > 0
        print(x, 'Success')
    except:
        print(row[1])
        q=input()

In [4]:
workdir = '/gws/nopw/j04/esacci_portal/kerchunk_conversion/'
with open(f'{workdir}/groups/cci_group_v1/proj_codes_continue.txt') as f:
    firstset = [r.strip() for r in f.readlines()]

with open(f'{workdir}/groups/cci_group_v1/proj_codes_continue2.txt') as f:
    secset = [r.strip() for r in f.readlines()]

newset = []
for s in secset:
    if s not in firstset:
        newset.append(s)

107

In [ ]:
newset

In [6]:
with open(f'{workdir}/groups/cci_group_v1/proj_codes_continue2.txt','w') as f:
    f.write('\n'.join(newset))

In [79]:
import xarray as xr
fileset = glob.glob('/neodc/esacci/aerosol/data/MS_UVAI/L3/v1.7/MONTHLY/*/ESACCI-AEROSOL-L3-AAI-MSAAI-1M-*-fv1.7.nc')
for f in fileset:
    ds = xr.open_dataset(f)
    print(ds.number_of_observations.shape)

(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)
(180, 360)